# Zalando scraper

In [247]:
## GOOD TO KNOW 

# website = https://zalando.nl/pre-owned-mode-heren/index.html
# article class = _0mW-4D _0xLoFW JT3_zV mo6ZnF _78xIQ- U4aOaA
# figure class = heWLCX
# tag product brand names = <span> 
# inspecting a product gives us: <a class="_LM JT3_zV CKDt_l CKDt_l LyRfpJ"href="https://www.zalando.nl/g-star-overhemd-blue-zir01hbly-001.html" rel="">


In [248]:
# Import packages
from bs4 import BeautifulSoup
import requests
import time 
from time import sleep
from datetime import datetime
import csv
import pandas as pd

# Make selenium and chromedriver work
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [249]:
#debug = {'verbose': sys.stderr}
user_agent = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36 Edg/98.0.1108.62'}  

In [250]:
# use webdriver.Chrome()
driver = webdriver.Chrome(ChromeDriverManager().install())

# run Zalando Pre-owned 
driver.get("https://www.zalando.nl/pre-owned-mode-heren/")
time.sleep(10)
print("done")




====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/sannevanettinger/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


done


In [251]:
# accept button on the website (close pop-ups before proceeding)

driver.find_elements_by_class_name("uc-btn-primary")

[<selenium.webdriver.remote.webelement.WebElement (session="5c9d19d0313708ef3a2cdc513f877452", element="04747abb-64c7-4bd2-a71d-82354f119270")>]

In [252]:
# Create a function to collect all page urls
base_url = "https://www.zalando.nl/pre-owned-mode-heren/?p="
counter = 1

# Now create the function
def generate_page_urls(base_url, num_pages):
    page_urls = []
    
    for counter in range (1, num_pages + 1):
        full_url = base_url + str(counter)
        page_urls.append(full_url)
    
    return page_urls

In [253]:
page_urls = generate_page_urls(base_url, 2) 

generate_page_urls("https://www.zalando.nl/pre-owned-mode-heren/?p=", 20)

['https://www.zalando.nl/pre-owned-mode-heren/?p=1',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=2',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=3',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=4',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=5',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=6',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=7',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=8',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=9',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=10',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=11',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=12',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=13',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=14',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=15',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=16',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=17',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=18',
 'https://www.zalando.nl/pre-owned-mo

In [254]:
# Create a function to collect the product_urls
def create_product_urls(page_urls):
    product_urls = []
    cnt=1
    for page_url in page_urls:
        cnt+=1
        driver.get(page_url)
        time.sleep(5)
        res = driver.page_source.encode('utf-8')
        soup = BeautifulSoup(res, "html.parser")
        soup.find(attrs={"data-trckng-component": "108K26XfnVjkToLFd4EmD"})
        print(len(products))
        sleep(1)

        for product in products:
            try:
                product_url = product.find("a").attrs["href"]
                print(product_url)
                product_urls.append(product_url)
            except:
                print("dit is geen product")
                
    return product_urls

In [255]:
product_urls = create_product_urls(page_urls)
len(product_urls)

85
https://www.zalando.nl/pure-overhemd-red-zir01d4f3-001.html
https://www.zalando.nl/hugo-overhemd-pink-zir01rxk1-001.html
https://www.zalando.nl/olymp-overhemd-grey-zir01rjtw-001.html
https://www.zalando.nl/anerkjendt-overhemd-green-zir01d4p5-001.html
https://www.zalando.nl/ben-sherman-vest-multi-colored-zir01qfya-001.html
https://www.zalando.nl/gaudi-vest-black-zir01sjqi-001.html
https://www.zalando.nl/john-richmond-vest-brown-zir01sjgq-001.html
https://www.zalando.nl/mads-noergaard-vest-blue-zir01scdb-001.html
https://www.zalando.nl/jack-and-jones-vest-blue-zir01skf8-001.html
https://www.zalando.nl/cadeaubonnen
https://www.zalando.nl/jack-and-jones-vest-brown-zir01qhtq-001.html
https://www.zalando.nl/boss-sweater-met-rits-grey-zir01sjur-001.html
https://www.zalando.nl/g-star-sweater-met-rits-grey-zir01t5tr-001.html
https://www.zalando.nl/jlindeberg-vest-blue-zir01qh50-001.html
https://www.zalando.nl/pierre-cardin-broek-green-zir01nw25-001.html
https://www.zalando.nl/hugo-broek-grey

170

In [256]:
scroll_counter = 0 
last_height = 0
driver.get("https://www.zalando.nl/pre-owned-mode-heren/")

# running this cell may take a minute or two
while True:    
    current_height = driver.execute_script('return document.body.scrollHeight')
    print("current height: " + str(current_height))

    if current_height == last_height:
        break

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    scroll_counter += 1
    last_height = current_height
    print("last height: " + str(last_height))
    sleep(1)
    
print(f"The number scrolls required to scrape all products is: {scroll_counter}")

current height: 8692
last height: 8692
current height: 8692
The number scrolls required to scrape all products is: 1


In [257]:
# scraping information from product pages
def get_product_pages(products):
    product_pages = []
    
    for url in product_urls: 
        res = requests.get(url, headers = user_agent)
        soup = BeautifulSoup(res.text, "html.parser")
        attributes = soup.find_all(class_="DT5BTM VHXqc_ rceRmQ _4NtqZU mIlIve")
        delivery = soup.find(class_="PBR5VH _0xLoFW EJ4MLB")
        
        for attribute in attributes:    
            product = attribute.find("h3", class_="mt1kvu ka2E9k uMhVZi FxZV-M SZKKsK pVrzNP _5Yd-hZ").text
            category = attribute.find("span", class_="EKabf7 R_QwOV").text
            price = attribute.find("span", class_="uqkIZw ka2E9k uMhVZi FxZV-M RYghuO pVrzNP")
            size = attribute.find("span", class_="_7Cm1F9 ka2E9k uMhVZi dgII7d RYghuO pVrzNP").text
            delivery_time = delivery.find("span", class_="u-6V88 ka2E9k uMhVZi dgII7d RYghuO pVrzNP").text
            delivery_time = delivery_time.replace("werkdagen", "")
            delivery_time = delivery_time.replace(" ", "")
            product_pages.append({"url": url,
                                 "brand": product,
                                 "category": category,
                                 "price": price,
                                 "size": size,
                                 "delivery time": delivery_time})
   
    return product_pages
print("done")

done


In [258]:
product_attributes = get_product_pages(product_urls)
product_attributes

AttributeError: 'NoneType' object has no attribute 'find'

In [87]:
from datetime import datetime

now = datetime.now()
print(now)

2022-03-15 19:42:40.492786


In [88]:
import csv 

with open("product_description_pre_owned.csv", "w", encoding = 'utf-8', newline='') as csv_file: 
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(["url", "brand", "category", "price", "size", "delivery time", "now"])
    now = datetime.now()
    for product in product_attributes: 
        writer.writerow([product['url'], product['brand'], product['category'], product['price'], product['size'], product['delivery time'], now])
print('done!')

done!
